# ***Clustering sobre Noticias de "El Tiempo" (Colombia)***

## ***Objetivo***

Identificar, analizar y resumir los temas predominantes en las noticias de "El Tiempo", con un enfoque en detectar patrones emergentes. A lo largo del ejercicio, se buscará la identificación de temas en busca de insights estratégicos sobre el interés público y los temas de impacto en Colombia.



## ***Pasos Detallados***

### ***1. Obtención de URLs de Noticias***
- **Uso de Web Scraping**: Recolecta URLs no solo de la sección de noticias recientes, sino de múltiples secciones como política, economía, deportes, y cultura.


### ***2. Extracción de resumenes del Texto de la Noticia***

- **Procesamiento Previo de Texto**: Realiza limpieza del texto (remoción de ruido, normalización, etc.).
- **Análisis de Sentimiento Inicial**: Como paso previo al clustering, utiliza una herramienta de análisis de sentimiento para agregar contexto emocional a los textos, lo cual puede enriquecer la interpretación de los clusters.

### 3. ***Generación de Embeddings con [`Ollama`](https://ollama.com/) y [`langchain`](https://python.langchain.com/docs/)***
- **Exploración Comparativa de Embeddings**: Genera embeddings utilizando diferentes modelos de Ollama para comparar la representatividad y consistencia de cada modelo en la agrupación de temas.
.

### ***4. Aplicación de Métodos de Clustering***
- **Exploración de Diferentes Algoritmos**: Experimenta con técnicas de clustering jerárquico, K-Means, GaussMixture.

- **Selección del Número de Clusters Dinámicamente**: Usa métodos como la silueta y Davies-Bouldin para determinar el número óptimo de clusters y realiza pruebas cruzadas.

- **Interpretación Visual**: Genera gráficos de reducción de dimensionalidad con UMAP y t-SNE para visualizar los clusters y detectar posibles temas subyacentes.

### ***5. Análisis con un LLM Local para Caracterización y Resumen de Clusters***

- **Uso de LLM para Insight Profundo**: Utiliza Ollama en Colab para generar resúmenes detallados y descripciones de cada cluster, incluyendo:

  - Resúmenes temáticos generales.
  - Identificación de términos clave.
  - Resumen de sentimientos predominantes dentro de cada cluster.


### ***6. Discusión y Evaluación de Resultados***

- **Contraste de Resultados con Noticias Internacionales**: Veirificar la perticencia de los clusters formados y realizar limpieza post modelado.
- **Generación de Reporte Visual e Interactivo**: Crea un tablero visual en Colab o herramientas como Streamlit para presentar los resultados, que incluya insights automáticos generados por el LLM para cada cluster.
- **Discusión y Conclusiones Estratégicas**: Propón hipótesis sobre por qué ciertos temas ganan o pierden relevancia en el periodo de análisis.


In [1]:
import requests
import pandas as pd
from io import StringIO

## ***Obtener urls de noticias del Tiempo***

Una sesión HTTP se es creada mediante `requests.Session()`. Las sesiones permiten mantener ciertas configuraciones y cookies a través de múltiples solicitudes.

In [2]:
url_new = (
    """https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar"""
    """-el-titulo-en-la-proxima-carrera-en-japon-3385519"""
)

# Crear una sesión HTTP para mantener cookies y configuraciones en múltiples solicitudes
session = requests.Session()
# Peticion HTTP
response = session.get(url_new)
# Respuesta peticion
print("Respuesta a la peticion url:", response.status_code)
print(f"{100 * '='}")
# Contenido de la pagina web
print(response.text)

Respuesta a la peticion url: 200
<!DOCTYPE html>
    <html lang="es">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
                                    

                                            
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<title>David Alonso sigue imparable en Moto3: ganó en Indonesia y podría asegurar el título en la próxima carrera, en Japón</title>
<link href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" rel="canonical">
<link rel="alternate" href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" hreflang="es-co"/>
<meta name

### ***¿Qué es un Sitemap XML?***
Un sitemap es un archivo en formato XML que contiene una lista de URLs de un sitio web. Su propósito principal es ayudar a los motores de búsqueda a encontrar, rastrear e indexar todas las páginas importantes del sitio de manera más eficiente. Esto es especialmente útil para sitios grandes o aquellos que actualizan contenido con frecuencia.

La URL https://www.eltiempo.com/sitemap-articles-current.xml apunta al sitemap de El Tiempo, un periódico digital. En este archivo, podemos encontrar las URLs de artículos recientes, que ayudan a que los motores de búsqueda como Google puedan descubrir e indexar rápidamente el contenido nuevo o actualizado.

In [3]:
url = 'https://www.eltiempo.com/sitemap-articles-current.xml'
session = requests.Session()
response = session.get(url)
if response.raise_for_status:
    print(f"OK, with the page: {url}. Status Code: {response.status_code}\n")
    print(f"Text from Page:")
    print(response.text[: 1000])
else:
    print(f"Problem in the page: {url}")

OK, with the page: https://www.eltiempo.com/sitemap-articles-current.xml. Status Code: 200

Text from Page:
<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:video="http://www.google.com/schemas/sitemap-video/1.1" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>https://www.eltiempo.com/mundo/eeuu-y-canada/cuantos-equipos-de-hockey-hay-en-florida-3394864</loc>
  <lastmod>2024-10-29T15:55:00-05:00</lastmod>
  <news:news>
   <news:publication>
    <news:name>ElTiempo.com</news:name>
    <news:language>es</news:language>
   </news:publication>
   <news:title>¿Cuántos equipos de hockey hay en Florida?</news:title>
   <news:publication_date>2024-10-29T15:55:00-05:00</news:publication_date>
   <news:keywords>cuántos, equipos, de, hockey, hay</news:keywords>
  </news:news>
 </url>
 <url>
  <loc>https://www.eltiempo.com/colombia/cali/cop16-supero-las-expectativas-secretaria-de-turismo-de-cali-sobre-los-mas-de-227-

El siguiente código extrae URLs de un sitemap XML y las convierte en un DataFrame de pandas para facilitar su análisis.

- Espacios de nombres XML: Define los espacios de nombres para identificar etiquetas en el XML:

- ns es el espacio principal del sitemap.

- news y video son espacios adicionales para etiquetas de noticias y videos.
- Lectura del XML:
    - pd.read_xml() lee el XML, convirtiendo el contenido en un DataFrame.
    - StringIO(response.text) convierte el contenido de response.text en un objeto similar a un archivo, que pd.read_xml() necesita para procesar el XML.
    - xpath=".//ns:url" selecciona las etiquetas <url> en el XML.
    - namespaces=namespaces asegura que los espacios de nombres definidos sean reconocidos.

- El DataFrame df_urls_news_el_tiempo contiene las URLs y sus datos asociados (URL, fecha de modificación, prioridad, etc.), que se muestran como una tabla en el notebook.

In [4]:
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "news": "http://www.google.com/schemas/sitemap-news/0.9",
    "video": "http://www.google.com/schemas/sitemap-video/1.1"
}

# Hacemos lectura del texto XML
df_urls_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//ns:url", namespaces=namespaces)
df_urls_news_el_tiempo

,loc,lastmod,news
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,2024-10-29T15:55:00-05:00,\n
1,https://www.eltiempo.com/colombia/cali/cop16-s...,2024-10-29T15:54:20-05:00,\n
2,https://www.eltiempo.com/bogota/en-vivo-movili...,2024-10-29T15:53:36-05:00,\n
3,https://www.eltiempo.com/mundo/eeuu-y-canada/q...,2024-10-29T15:53:18-05:00,\n
4,https://www.eltiempo.com/bogota/me-mato-a-mis-...,2024-10-29T15:50:49-05:00,\n
...,...,...,...
8396,https://www.eltiempo.com/mundo/medio-oriente/p...,2024-10-01T00:00:00-05:00,\n
8397,https://www.eltiempo.com/mas-contenido/del-cru...,2024-10-01T00:00:00-05:00,\n
8398,https://www.eltiempo.com/mas-contenido/el-impo...,2024-10-01T00:00:00-05:00,\n
8399,https://www.eltiempo.com/mas-contenido/los-tri...,2024-10-01T00:00:00-05:00,\n


In [5]:
# Hacemos lectura del texto XML
df_title_date_keywords_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//news:news", namespaces=namespaces)
df_title_date_keywords_news_el_tiempo

,publication,title,publication_date,keywords
0,\n,¿Cuántos equipos de hockey hay en Florida?,2024-10-29T15:55:00-05:00,"cuántos, equipos, de, hockey, hay"
1,\n,COP16: 'Esto ha sido un gran éxito para nuestr...,2024-10-29T15:54:20-05:00,"COP16, Zona, Verde, Cali"
2,\n,EN VIVO | Movilidad en Bogotá 29 de octubre: 2...,2024-10-29T15:53:36-05:00,"Movilidad, Bogotá, Tránsito, Tráfico, Lluvias,..."
3,\n,"¿Quién ganará las elecciones de EE. UU., según...",2024-10-29T15:53:18-05:00,"quién, ganará, las, elecciones, de"
4,\n,'Me mató a mis niños': el crudo testimonio de ...,2024-10-29T15:50:49-05:00,"Bogotá, Las Ferias, Asesinato, Filicidio, Darw..."
...,...,...,...,...
8396,\n,Crisis en Oriente Medio: ¿por qué Israel lanzó...,2024-10-01T00:00:00-05:00,"Israel, Hezbolá, Líbano, Incursión terrestre, ..."
8397,\n,Del crudo al turismo,2024-10-01T00:00:00-05:00,"del, crudo, al, turismo"
8398,\n,El imponente tesoro verde,2024-10-01T00:00:00-05:00,"el, imponente, tesoro, verde"
8399,\n,Los trinos que mejor silban en la sabana,2024-10-01T00:00:00-05:00,"los, trinos, que, mejor, silban"


In [6]:
df_urls_news_el_tiempo = (
    df_urls_news_el_tiempo[["loc"]]
    .merge(
        df_title_date_keywords_news_el_tiempo.drop(columns=["publication"]),
        left_index=True,
        right_index=True        
    )
    .rename(columns={"loc": "url_page"})
)

df_urls_news_el_tiempo

,url_page,title,publication_date,keywords
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,¿Cuántos equipos de hockey hay en Florida?,2024-10-29T15:55:00-05:00,"cuántos, equipos, de, hockey, hay"
1,https://www.eltiempo.com/colombia/cali/cop16-s...,COP16: 'Esto ha sido un gran éxito para nuestr...,2024-10-29T15:54:20-05:00,"COP16, Zona, Verde, Cali"
2,https://www.eltiempo.com/bogota/en-vivo-movili...,EN VIVO | Movilidad en Bogotá 29 de octubre: 2...,2024-10-29T15:53:36-05:00,"Movilidad, Bogotá, Tránsito, Tráfico, Lluvias,..."
3,https://www.eltiempo.com/mundo/eeuu-y-canada/q...,"¿Quién ganará las elecciones de EE. UU., según...",2024-10-29T15:53:18-05:00,"quién, ganará, las, elecciones, de"
4,https://www.eltiempo.com/bogota/me-mato-a-mis-...,'Me mató a mis niños': el crudo testimonio de ...,2024-10-29T15:50:49-05:00,"Bogotá, Las Ferias, Asesinato, Filicidio, Darw..."
...,...,...,...,...
8396,https://www.eltiempo.com/mundo/medio-oriente/p...,Crisis en Oriente Medio: ¿por qué Israel lanzó...,2024-10-01T00:00:00-05:00,"Israel, Hezbolá, Líbano, Incursión terrestre, ..."
8397,https://www.eltiempo.com/mas-contenido/del-cru...,Del crudo al turismo,2024-10-01T00:00:00-05:00,"del, crudo, al, turismo"
8398,https://www.eltiempo.com/mas-contenido/el-impo...,El imponente tesoro verde,2024-10-01T00:00:00-05:00,"el, imponente, tesoro, verde"
8399,https://www.eltiempo.com/mas-contenido/los-tri...,Los trinos que mejor silban en la sabana,2024-10-01T00:00:00-05:00,"los, trinos, que, mejor, silban"


## ***Obtener el texto de las noticias***

In [7]:
from bs4 import BeautifulSoup

In [8]:
# Ejemplo de url a visitar:
df_urls_news_el_tiempo.iloc[0].loc["url_page"]

'https://www.eltiempo.com/mundo/eeuu-y-canada/cuantos-equipos-de-hockey-hay-en-florida-3394864'

In [9]:
%%time
# Primeras 20 urls de noticias.
list_urls = df_urls_news_el_tiempo["url_page"].head(5).to_list()

for url in list_urls:
    print("Trabajando en la URL:", url)
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    # Verificar si el request es exitosa
    if request.raise_for_status:
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])     
        print(f"Autor: {author_tag}")
        print(f"Contenido noticia: {content}")
        print()

Trabajando en la URL: https://www.eltiempo.com/mundo/eeuu-y-canada/cuantos-equipos-de-hockey-hay-en-florida-3394864
Autor: Joaquin Corbetta
Contenido noticia: La temporada regular de la Liga Nacional de Hockey sobre hielo (NHL, por sus siglas en inglés) comenzó a principios de octubre y se extenderá hasta abril. La liga está compuesta por 32 equipos, al igual que en la NBA, dividida en dos conferencias: Este y Oeste. En el este hay dos equipos de Florida que disputan la Stanley Cup. Si bien los equipos están en la misma conferencia, se encuentra en distintas divisiones, pero compiten entre sí. Estos son los 2 equipos de Florida: Florida Panthers: Los Florida Panthers disputaron su primera temporada en 1993, tras coquetear con el título en su tercera temporada, tan solo ganaron una serie de playoffs en los 2022. Actualmente, se encuentran liderando la conferencia este con 11 partidos disputados y 7 ganamos.Tampa Bay Lightning: El equipo ha sido históricamente uno de los mejores de la li

## ***Paralelizar la tarea de obtencion del contenido de las noticias***

In [10]:
session = requests.Session()

def get_content_news_from_url(url: str) -> dict:
    """
    Obtiene el contenido de una noticia desde una URL, extrayendo el autor y el cuerpo de la noticia.

    :param url: URL de la página de la noticia.
    :return: Diccionario con la URL, autor y contenido de la noticia.
    :raises ValueError: Si hay un problema con el status_code de la solicitud.
    """
    request = session.get(url)
    
    if request.raise_for_status:
        soup = BeautifulSoup(request.content, "html.parser")
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])
        
        return {
            "url_page": url,
            "autor": author_tag,
            "news_content": content            
        }
    else:
        ValueError(
            f"Problemas con el status_code: {request.status_code}"
        )
        

get_content_news_from_url(df_urls_news_el_tiempo.iloc[0].loc["url_page"]) 

{'url_page': 'https://www.eltiempo.com/mundo/eeuu-y-canada/cuantos-equipos-de-hockey-hay-en-florida-3394864',
 'autor': 'Joaquin Corbetta',
 'news_content': 'La temporada regular de la Liga Nacional de Hockey sobre hielo (NHL, por sus siglas en inglés) comenzó a principios de octubre y se extenderá hasta abril. La liga está compuesta por 32 equipos, al igual que en la NBA, dividida en dos conferencias: Este y Oeste. En el este hay dos equipos de Florida que disputan la Stanley Cup. Si bien los equipos están en la misma conferencia, se encuentra en distintas divisiones, pero compiten entre sí. Estos son los 2 equipos de Florida: Florida Panthers: Los Florida Panthers disputaron su primera temporada en 1993, tras coquetear con el título en su tercera temporada, tan solo ganaron una serie de playoffs en los 2022. Actualmente, se encuentran liderando la conferencia este con 11 partidos disputados y 7 ganamos.Tampa Bay Lightning: El equipo ha sido históricamente uno de los mejores de la lig

In [11]:
import os
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

num_cores = os.cpu_count()
print(f"Número de cores disponibles: {num_cores}")

# Es recomendado dejar un core libre
# num_cores = num_cores - 1


def parallel_process_urls(urls: list[str]) -> list[Dict]:
    """
    Procesa múltiples URLs en paralelo utilizando ThreadPoolExecutor,
    dejando 2 cores libres.
    
    :param urls: Lista de URLs a procesar.
    :return: Lista de diccionarios con la información extraída de cada URL.
    """
    results = []
    
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        futures = [executor.submit(get_content_news_from_url, url) for url in urls]
        
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error procesando la URL: {e}")
    
    return results

# Lista de URLs a procesar. Descomente la siguiente linea de codigo 
# para obtener todas las urls.
# Procesamos todas las URLs en paralelo
# urls_to_process = df_urls_news_el_tiempo["url_page"].tolist()

# urls de ejemplo, solo una muestra de 20 urls
urls_to_process = df_urls_news_el_tiempo["url_page"].sample(20).tolist()
# Procesamos todas las URLs en paralelo
processed_results = parallel_process_urls(urls_to_process)
# Resultados
print(processed_results)
pd.DataFrame(processed_results)

Número de cores disponibles: 16
[{'url_page': 'https://www.eltiempo.com/vida/ciencia/eclipse-solar-anular-2024-estos-son-los-paises-que-podran-observar-el-evento-astronomico-de-este-2-de-octubre-3386270', 'autor': 'Alejandra López Plazas', 'news_content': "Un eclipse es un fenómeno astronómico que sucede cuando la Luna se ubica entre el Sol y la Tierra. La sombra de la Luna (umbra) se proyecta sobre la Tierra, dibujando una franja donde el eclipse puede verse total o anular. Un observador fuera de la franja de totalidad o anularidad verá el eclipse de Sol de manera parcial. (Le puede interesar: Científicos descubren nueva forma de crear diamantes en laboratorio en menos de 3 horas) El día de mañana, algunos países de Suramérica podrán apreciar este fenómeno de manera anular en la que la Luna cubre el centro del Sol, dejando que los bordes exteriores visibles del Sol formen un 'anillo de fuego'. Este tipo de eclipse sucede cuando la Luna está más lejos de la Tierra que lo habitual, es d

,url_page,autor,news_content
0,https://www.eltiempo.com/vida/ciencia/eclipse-...,Alejandra López Plazas,Un eclipse es un fenómeno astronómico que suce...
1,https://www.eltiempo.com/colombia/barranquilla...,Leonardo Herrera Delgans,"El Instituto de Hidrología, Meteorología y Est..."
2,https://www.eltiempo.com/mundo/europa/rusia-co...,efe,La Justicia rusa condenó este lunes a seis año...
3,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,Tomas Peiro,De cara a las elecciones presidenciales de Est...
4,https://www.eltiempo.com/economia/sectores/the...,Mauricio Galindo,"""President,"" said the Minister of Environment,..."
5,https://www.eltiempo.com/colombia/barranquilla...,Deivis Jhoan López Ortega,A unos días de cumplirse el primer mes desde q...
6,https://www.eltiempo.com/cultura/gente/realmen...,Wendys Loraine Pitre Ariza,Existe una creencia que ha ido de generación e...
7,https://www.eltiempo.com/cultura/gente/el-cart...,Erika Lucia Ibañez García,En la búsqueda de alimentos que contribuyan a ...
8,https://www.eltiempo.com/mundo/latinoamerica/l...,Francisco Barbosa,Golpes blandos y lawfare son las palabras favo...
9,https://www.eltiempo.com/cultura/gente/mujer-d...,Noticias GDA,"Una residente de Florida, conocida en TikTok b..."


In [12]:
df_news_el_tiempo = df_urls_news_el_tiempo.merge(
    pd.DataFrame(processed_results),
    on=["url_page"]    
)

display(df_news_el_tiempo)
# Descomente las siguientes lineas de codigo se desea guarda los resultados
# en un archivo parquet
# df_news_el_tiempo.to_parquet("data/df_noticias_el_tiempo.parquet")
# pd.read_parquet("data/df_noticias_el_tiempo.parquet")

,url_page,title,publication_date,keywords,autor,news_content
0,https://www.eltiempo.com/mundo/eeuu-y-canada/t...,Texas: entregan despensas gratis en estas zona...,2024-10-25T12:02:04-05:00,"texas, darán, despensas, gratis, en",Joaquin Corbetta,Debido al contexto inflacionario y la crisis e...
1,https://www.eltiempo.com/cultura/gente/el-cart...,El cartel de los alimentos que más disparan el...,2024-10-23T14:46:13-05:00,"el, cartel, de, los, alimentos",Erika Lucia Ibañez García,En la búsqueda de alimentos que contribuyan a ...
2,https://www.eltiempo.com/economia/sectores/the...,"The largest debt swap in the world by nature, ...",2024-10-23T12:01:08-05:00,"the, largest, debt, swap, in",Mauricio Galindo,"""President,"" said the Minister of Environment,..."
3,https://www.eltiempo.com/politica/gobierno/min...,"Ministro del Interior, Juan Fernando Cristo, i...",2024-10-19T11:36:41-05:00,"Gustavo Petro, CNE",Mateo García Agudelo,Casi 23 horas después de que se conoció la res...
4,https://www.eltiempo.com/mundo/latinoamerica/m...,Mujer se resbaló mientras limpiaba una ventana...,2024-10-17T13:57:48-05:00,"Video, Mujer, Resbala, Limpia, Ventana, Edificio",Noticias GDA,"Un impresionante incidente en Guarujá, São Pau..."
5,https://www.eltiempo.com/mundo/latinoamerica/l...,"La batalla jurídica en Latinoamérica, entre la...",2024-10-17T11:58:04-05:00,"la, batalla, jurídica, en, latinoamérica",Francisco Barbosa,Golpes blandos y lawfare son las palabras favo...
6,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,'La tormenta más grande del sistema solar': ci...,2024-10-17T11:05:58-05:00,"la, tormenta, más, grande, del",Donato Del Blanco,"La Gran Mancha Roja de Júpiter, conocida como ..."
7,https://www.eltiempo.com/deportes/futbol-colom...,Video | Con una mala noche y una nueva polémic...,2024-10-13T00:00:00-05:00,"Equidad, Junior, Liga BetPlay",José Orlando Ascencio,"junior sigue lleno de dudas, jugando muy mal y..."
8,https://www.eltiempo.com/cultura/gente/mujer-d...,Mujer de Florida explica por qué su familia se...,2024-10-09T08:26:11-05:00,"Mujer, Florida, Casa, Familia, Huracán Milton,...",Noticias GDA,"Una residente de Florida, conocida en TikTok b..."
9,https://www.eltiempo.com/cultura/musica-y-libr...,"Organizadores de Comunion, festival de música ...",2024-10-09T00:00:00-05:00,"se, cancela, el, festival, de",Juan Jose Rios Arbelaez,"Los organizadores de Comunion, el festival de ..."


## ***Usar un LLM local***

In [13]:
# Instalar Ollama (disponible para Windows, Linux y MacOS)
# Aca se debe poner la guia de instalacion en Colab (pendiente)
# 1. Instalemos Ollama en Colab --> !curl -fsSL https://ollama.com/install.sh | sh
# 2. Verificamos su instalacion --> !ollama --version
# 3. Instalamos una terminal en Colab para ejecutar Ollama
#       !pip install colab-xterm
#       %load_ext colabxterm
#       %xterm
# 4. Servir Ollama: run en terminal ---> ollama serve
# 5. Descargar modelo: !ollama pull qwen2.5:7b
# 6. Probar modelo: 
#       %xterm
#       !ollama run qwen2.5:7b

In [14]:
# !curl -fsSL https://ollama.com/install.sh | sh

In [15]:
# !ollama --version

In [16]:
## Installemos una terminal en Colab para ejecutar Ollama
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm
##run in terminal ---> ollama serve

In [17]:
# !ollama pull qwen2.5:7b

In [18]:
# %xterm
# ollama run qwen2.5:7b

In [19]:
#!pip install langchain-ollama==0.2.0

In [20]:
# Generar un chat con un llm_ollama
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=1024,
)

llm_response = llm_ollama.invoke("¿Cuál es la segunda letra del alfabeto griego?")
print(llm_response)

content='La segunda letra del alfabeto griego es "Beta" (Β ηβ), que se escribe así en mayúscula y así en minúscula.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-29T21:36:22.399663432Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 13504816316, 'load_duration': 12219485212, 'prompt_eval_count': 43, 'prompt_eval_duration': 120293000, 'eval_count': 38, 'eval_duration': 1000284000} id='run-2a738f28-c129-404c-9d0f-7a54eb4513c6-0' usage_metadata={'input_tokens': 43, 'output_tokens': 38, 'total_tokens': 81}


In [21]:
%%time
llm_ollama.invoke(
    """Dame un lista de 20 personajes influyentes en la actualidad"""
    """mundial. No olvides incluir diferentes angulos de la vida. """
    """Solo debes darme la lista sin decir nada mas. Por ejemplo si """
    """decides incluir a Pepito Perez (suponiendo que es influyente """
    """actualmente) respuesta debe ser: Pepito Perez"""
    )

CPU times: user 115 ms, sys: 21 ms, total: 136 ms
Wall time: 2.57 s


AIMessage(content='Bill Gates\nElon Musk\nAngela Merkel\nJoe Biden\nXi Jinping\nBoris Johnson\nBernie Madoff\nGreta Thunberg\nOprah Winfrey\nJack Ma\nMark Zuckerberg\nTaylor Swift\nLeBron James\nCristiano Ronaldo\nNeymar Jr.\nShakira\nAmma Asante\nKanye West\nPriyanka Chopra Jonas\nBillie Eilish', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-29T21:36:25.039088029Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2540320943, 'load_duration': 110129329, 'prompt_eval_count': 102, 'prompt_eval_duration': 134917000, 'eval_count': 87, 'eval_duration': 2137488000}, id='run-6885bc3a-aeaf-4b67-a3c7-80817bd24240-0', usage_metadata={'input_tokens': 102, 'output_tokens': 87, 'total_tokens': 189})

In [22]:
# PromptTemplate y respuesta como string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida en diferentes aspectos. Responde usando solo JSON con """
            """llave el nombre del personaje y valor su biografia, """
            """Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))"""),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

print(prompt)

chain = prompt | llm_ollama | StrOutputParser()
chain.invoke(
    {
        "user_input": "Barack Obama"
    }
)

input_variables=['user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres una util AI bot que crea excelentes biografias de personajes influyentes de la actualidad mundial y detalla su vida en diferentes aspectos. Responde usando solo JSON con llave el nombre del personaje y valor su biografia, Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


'{\n  "Barack Obama": "Barack Obama es un político estadounidense que sirvió como el 44º presidente de los Estados Unidos desde 2009 hasta 2017. Nació en Honolulu, Hawái, el 4 de agosto de 1961, y fue educado en Hawaii, Indonesia y Columbia. Antes de su presidencia, Obama trabajó como abogado y legislador estatal en Illinois. Fue elegido senador por Illinois en 2004 y luego se convirtió en el primer afroamericano presidente de los Estados Unidos tras ganar las elecciones presidenciales de 2008. Durante su mandato, Obama implementó reformas significativas en áreas como la salud, la educación y el cambio climático. Después de dejar la presidencia, ha seguido siendo una figura influyente a nivel mundial, participando activamente en causas humanitarias y políticas internacionales."\n}'

In [23]:
# Usamos JSON-format para parsear la respuesta del LLM
from langchain_core.output_parsers import JsonOutputParser

json_llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=-1,
    format="json"
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida ampliamente en diferentes aspectos. No debes escatimar """
            """en la longitud del texto de resumen. Responde usando solo JSON """
            """format con llave el nombre del personaje y valor su biografia"""
            """Por ejemplo: dict(Pepito Perez="Pepito Perez ... (biografia)","""
            """ Benito Camelas="Benito Camelas ... (biografia)"). Debes """
            """seguir al pie de la letra el formato"""
            ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = prompt | json_llm | JsonOutputParser()
llm_answer = chain.invoke({"user_input": "Dame la biografia de 2 personajes"})
print(llm_answer, type(llm_answer))

{'Bill Gates': "Bill Gates, nacido el 28 de octubre de 1955 en Seattle, Washington, es un empresario, filántropo y escritor estadounidense. Es co-fundador de Microsoft Corporation junto con Paul Allen, donde ocupó la posición de presidente hasta 1981 y luego como director ejecutivo hasta 2000. Gates se retiró oficialmente de Microsoft en 2008 pero sigue siendo un asesor estratégico del consejo de administración. En 2000, junto con su esposa Melinda, fundó la Fundación Bill & Melinda Gates, una organización sin fines de lucro dedicada a mejorar las condiciones de vida en el mundo, especialmente en áreas como la salud y la educación. La fundación se ha convertido en una de las más grandes del mundo en términos de donaciones. Además de su trabajo filantrópico, Gates es autor de varios libros, incluyendo 'Business @ the Speed of Thought' y 'The Road Ahead', que exploran el impacto de la tecnología en la sociedad. Su interés por la educación también se manifiesta a través del Instituto de T

### ***Embeddings***

In [24]:
# !ollama pull mxbai-embed-large:latest

In [25]:
%%time
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

question = ["¿Cuál es la segunda letra del alfabeto griego?"]
embed_question = ollama_embeddings.embed_documents(question)
print("Embeddings:")
pd.DataFrame(embed_question, index=question)

Embeddings:
CPU times: user 424 ms, sys: 28.4 ms, total: 452 ms
Wall time: 4.88 s


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
¿Cuál es la segunda letra del alfabeto griego?,-0.04817,0.015093,0.041272,0.001454,-0.037321,-0.011506,0.014483,0.003339,0.024329,-0.007726,...,0.045982,-0.015122,0.012864,-0.031581,0.009468,0.03363,-0.001437,0.034952,-0.060254,-0.037693


In [26]:
# Ejemplo: (Relevancia - Distancia)
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

texts = [
    "Alpha es la primera letra del alfabeto griego",
    "A es la primera letra del alfabeto latino",
    "Beta es la segunda letra del alfabeto griego",
    "B es la segunda letra del alfabeto latino"
]
query = ["¿Cuál es la segunda letra del alfabeto griego?"]
embeds = ollama_embeddings.embed_documents(texts)
embed_query = ollama_embeddings.embed_documents(query)
print(f"Long Ollama Embeddings: {len(embed_query[0])}")
print(f"Cosine Similarities: \n {cosine_similarity(embeds, embed_query)}")

pd.DataFrame(
    cosine_similarity(embeds, embed_query),
    index=texts,
    columns=query
).sort_values(by=query, ascending=False)

Long Ollama Embeddings: 1024
Cosine Similarities: 
 [[0.76798198]
 [0.77720013]
 [0.89850202]
 [0.8555794 ]]


,¿Cuál es la segunda letra del alfabeto griego?
Beta es la segunda letra del alfabeto griego,0.898502
B es la segunda letra del alfabeto latino,0.855579
A es la primera letra del alfabeto latino,0.777200
Alpha es la primera letra del alfabeto griego,0.767982


In [27]:
def get_summary_news(news: str, model_name: str = "qwen2.5:7b") -> str:

    llm_ollama = ChatOllama(
        model=model_name,
        temperature=0.1,
        num_predict=4096,
        num_ctx=16000,
        keep_alive=0
    )

    prompt_summarization = ChatPromptTemplate(
        [
            SystemMessagePromptTemplate.from_template(
                """Por favor, crea un resumen natural, claro y preciso de esta noticia. Asegúrate de incluir los 
                siguientes puntos de manera fluida::"""
                """1. Evento principal o tema central: Describe en una o dos frases el evento clave o el tema más """
                """importante de la noticia."""
                """2. Actores principales y sus reacciones: Menciona las partes involucradas y sus posturas o 
                reacciones significativas (gobiernos, empresas, expertos, etc.)."""
                """3. Consecuencias o implicaciones: Señala las implicaciones principales o cualquier posible """
                """consecuencia a corto o largo plazo derivada de la noticia."""
                """4. Contexto adicional relevante: Incluye cualquier información de contexto que sea relevante para """
                """comprender mejor el evento o las decisiones tomadas."""
                """El resumen debe ser natural y no exceder las cinco frases. Evita juicios o interpretaciones, 
                    enfocándote en capturar la esencia de la noticia sin perder detalles importantes."""
            ),
            HumanMessagePromptTemplate.from_template("El texto de la noticias es:\n{news}")
        ]
    )

    llm_chain = prompt_summarization | llm_ollama | StrOutputParser()

    return llm_chain.invoke({"news": news})

In [29]:
df_test = df_news_el_tiempo.sample(5)
df_test = df_test[["url_page", "news_content"]]
df_test["summary"] = df_test["news_content"].apply(get_summary_news)
df_test

,url_page,news_content,summary
2,https://www.eltiempo.com/economia/sectores/the...,"""President,"" said the Minister of Environment,...",El tema central es el acuerdo de intercambio d...
18,https://www.eltiempo.com/vida/ciencia/eclipse-...,Un eclipse es un fenómeno astronómico que suce...,Un eclipse solar anular será visible mañana en...
13,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,De cara a las elecciones presidenciales de Est...,Joe Biden ha mantenido un perfil bajo en la ca...
1,https://www.eltiempo.com/cultura/gente/el-cart...,En la búsqueda de alimentos que contribuyan a ...,El tema central de la noticia es la importanci...
3,https://www.eltiempo.com/politica/gobierno/min...,Casi 23 horas después de que se conoció la res...,El Consejo Nacional Electoral (CNE) abrió form...


In [30]:
for _, row in df_test.iterrows():
    print(f"URL: {row['url_page']}")
    print(f"Noticia: {row['news_content']}")
    print(f"Resumen: {row['summary']}\n")

URL: https://www.eltiempo.com/economia/sectores/the-largest-debt-swap-in-the-world-by-nature-a-relief-of-us-1-1-billion-told-by-former-ecuadorian-foreign-minister-gustavo-manrique-protagonist-of-the-agreement-3392966
Noticia: "President," said the Minister of Environment, Gustavo Manrique to the President of Ecuador, Guillermo Lasso, "do you want to save more than a billion dollars of debt?" (Lea esta historia en español, aquí) It was 2022, and Manrique was referring to a way to achieve this relief in his country's public finances by taking advantage of the value of the diversity of life in its territory. And to convince Lasso, he says now, he could have resorted to explanations about the importance of certain species in the food chain, or talked about CO2 sequestration. Instead, he used a more universally understood language.... "Do you want to save over a billion dollars in debt?" And the conversation and the president's positive response, Manrique says, lasted three minutes, and the

In [31]:
# Embedding de las noticias.
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

embed_news = ollama_embeddings.embed_documents(df_test["summary"].to_list())
print("Embeddings:")
pd.DataFrame(embed_news, index=df_test["url_page"])

Embeddings:


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
url_page,,,,,,,,,,,,,,,,,,,,,
https://www.eltiempo.com/economia/sectores/the-largest-debt-swap-in-the-world-by-nature-a-relief-of-us-1-1-billion-told-by-former-ecuadorian-foreign-minister-gustavo-manrique-protagonist-of-the-agreement-3392966,0.026223,0.037938,-0.022923,0.015873,-0.010306,0.022605,-0.014388,0.029748,0.032824,0.046142,...,0.025176,0.001309,-0.038515,-0.005819,0.037186,-0.008255,-0.040583,0.001247,-0.040471,0.014252
https://www.eltiempo.com/vida/ciencia/eclipse-solar-anular-2024-estos-son-los-paises-que-podran-observar-el-evento-astronomico-de-este-2-de-octubre-3386270,-0.000772,0.043617,0.003502,-0.026757,-0.017752,0.004504,-0.010096,0.033927,0.024615,0.046132,...,0.031555,-0.025900,-0.024077,-0.003740,0.030551,0.024922,0.012350,-0.000842,-0.036382,-0.002661
https://www.eltiempo.com/mundo/eeuu-y-canada/la-sorpresiva-actitud-de-joe-biden-en-la-campana-de-kamala-harris-3387928,-0.016373,-0.030462,0.018463,0.037656,0.024227,0.028127,-0.067299,0.039541,0.005734,-0.016572,...,0.027244,0.033238,-0.055575,-0.056069,0.050637,0.011347,-0.043135,-0.014779,-0.047541,-0.038939
https://www.eltiempo.com/cultura/gente/el-cartel-de-los-alimentos-que-mas-disparan-el-azucar-en-sangre-hay-hasta-frutas-en-la-lista-3393003,-0.011063,0.016162,-0.009992,0.044199,-0.049189,-0.027564,0.011281,0.031589,0.009738,0.048391,...,0.072141,0.003308,-0.015579,-0.013793,0.026030,0.014966,-0.055860,-0.004046,-0.013482,0.012700
https://www.eltiempo.com/politica/gobierno/ministro-del-interior-juan-fernando-cristo-insiste-en-que-el-cne-no-tiene-facultades-para-investigar-al-presidente-petro-3391763,-0.009061,-0.015961,-0.012615,-0.004264,0.016769,0.024066,-0.079951,0.029291,-0.004389,0.032306,...,0.060231,0.009458,-0.022950,-0.035527,0.046973,0.010659,-0.058113,0.000607,-0.033516,0.011918
